In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import zeus.notebook_utils.syspath as syspath
syspath.add_parent_folder()

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
from zeus.utils import list_files
from kidney.datasets.kaggle import get_reader, SampleType
from kidney.inference.prediction import read_predictions, MajorityVotePrediction
from kidney.utils.mask import rle_decode, rle_numba_encode

In [ ]:
PREDICTIONS_DIR = os.path.join(os.environ["DATASET_ROOT"], "predictions")

In [ ]:
reader = get_reader()

In [ ]:
[dirname] = list_files(PREDICTIONS_DIR)

In [ ]:
test_keys = reader.get_keys(SampleType.Unlabeled)

In [ ]:
predictions = read_predictions(dirname).to_dict("index")

In [ ]:
encoded_masks = []
n = len(test_keys)
for i, key in enumerate(test_keys):
    print(f"Predicting key {key} ({i+1} of {n})")
    sample = reader.fetch_one(key)
    image = sample["image"]
    mask_size = image.shape[:2]
    predictor = MajorityVotePrediction(
        predictions=predictions,
        mask_size=mask_size,
        majority=0.5
    )
    mask_pred = predictor(key)
    encoded = rle_numba_encode(mask_pred)
    encoded_masks.append({"id": key, "predicted": encoded})

In [ ]:
encoded_masks_df = pd.DataFrame(encoded_masks)

In [ ]:
# !ls /mnt/fast/kaggle/submits/kidney

In [ ]:
experiment_name = "smp_upp_effnet_b4_dice_1c_o256_folds_4_maj_05"

In [ ]:
encoded_masks_df.to_csv(f"/mnt/fast/kaggle/submits/kidney/{experiment_name}.csv", index=False)